In [ ]:
# Imports
import os
import pandas as pd
import json
from zipfile import ZipFile
import requests
import shutil

# Nous vérifions si le fichier existe ou non avant de le supprimer.
if os.path.exists('C:/Users/carla/Desktop/telechargements/data.zip'):
    os.remove('C:/Users/carla/Desktop/telechargements/data.zip')
else:
    print("Impossible de supprimer le fichier car il n'existe pas")
# Nous effaçons le dossier et tous les fichiers
try :
  shutil.rmtree("C:/Users/carla/Desktop/dezipage/data_unzipped", ignore_errors=True)
except:
  pass


# Téléchargement du fichier dans le dossier "telechargements"
url ="https://diffuseur.datatourisme.gouv.fr/webservice/024d893f2af1b82ab4ca5d66dec6df0c/cd13e69e-874b-4eaf-ac53-8f5c8680fec0"
r = requests.get(url, r'C:/Users/carla/Desktop/telechargements/data.zip')
with open(r'C:/Users/carla/Desktop/telechargements/data.zip', 'wb') as f:
    f.write(r.content)
    
# dezipage des fichiers dans le dossier "dezipage"
with ZipFile(r'C:/Users/carla/Desktop/telechargements/data.zip') as zipObj:
    zipObj.extractall(path='C:/Users/carla/Desktop/dezipage/data_unzipped', members=None, pwd=None)

     
# Chemin d'accès du fichier INDEX qui regroupe tous les POI
index_json_file_path = 'C:/Users/carla/Desktop/dezipage/data_unzipped/index.json'

# Overture et lecture du fichier INDEX
with open(index_json_file_path, 'r', encoding='utf8', errors='ignore') as j:
     contents = json.loads(j.read())
        

# Création de la liste de tous les chemins d'accès de tous les fichiers JSON
json_files_path = []

# Boucle pour avoir tous les chemins d'accès de tous les fichiers JSON
for i in range(len(contents)):
    json_file_subdirectory = contents[i]['file']
    complete_json_file_directory = 'C:/Users/carla/Desktop/dezipage/data_unzipped/objects/' + json_file_subdirectory
    json_files_path.append(complete_json_file_directory)


# Création des listes
#labels = []
tags_list = []
cities = []
#postal_codes = []
#codes_insee =[]
lats=[]
longs=[]
#departements=[]
regions=[]
code_regions=[]
office_tourismes=[]
fournisseurs=[]
ids_poi=[]
photos=[]


# Boucle pour récupérer toutes les informations qu'on souhaite dans les fichiers JSON
for json_file_path in json_files_path:
    
        with open(json_file_path, 'r',encoding='utf8', errors='ignore') as j:
            contents = json.loads(j.read())
    
        #try:
            #label = contents['rdfs:label']['fr'][0]
            #labels.append(label)
        #except:
            #labels.append('NaN')

        try:
            tags = contents['@type']
            tags_list.append(tags)
        except:
            tags_list.append('NaN')

        try:
            city = contents['isLocatedAt'][0]['schema:address'][0]['schema:addressLocality']
            cities.append(city)
        except:
            cities.append('NaN')

        #try:
           # postal_code = contents['isLocatedAt'][0]['schema:address'][0]['schema:postalCode']
           # postal_codes.append(postal_code)
       # except:
          #  postal_codes.append('NaN')
        #    
        #try:
           ## code_insee= contents['isLocatedAt'][0]['schema:address'][0]['hasAddressCity']['insee']
           # codes_insee.append(code_insee)
       # except:
           # codes_insee.append('NaN')
            
        try:
            lat= contents['isLocatedAt'][0]['schema:geo']['schema:latitude']
            lats.append(lat)
        except:
            lats.append('NaN')
            
        try:
            long= contents['isLocatedAt'][0]['schema:geo']['schema:longitude']
            longs.append(long)
        except:
            longs.append('NaN')
        
        #    
        #try:
            #departement = contents['isLocatedAt'][0]['schema:address'][0]['hasAddressCity']['isPartOfDepartment']['rdfs:label']['fr']
            #departements.append(departement)
        
       # except:
           # departements.append('NaN')
            
        try:
            region = contents['isLocatedAt'][0]['schema:address'][0]['hasAddressCity']['isPartOfDepartment']['isPartOfRegion']['rdfs:label']['fr']
            regions.append(region)
        except:
            regions.append('NaN')
        
        try:
            code_region = contents['isLocatedAt'][0]['schema:address'][0]['hasAddressCity']['isPartOfDepartment']['insee']
            code_regions.append(code_region)
        except:
            code_regions.append('NaN')
        
        try:
            office_tourisme= contents['hasBeenCreatedBy']['schema:legalName']
            office_tourismes.append(office_tourisme)
        except:
            office_tourismes.append('NaN')
            
        try:
            fournisseur= contents['hasBeenPublishedBy'][0]['schema:legalName']
            fournisseurs.append(fournisseur)
        except: 
            fournisseurs.append('NaN')
            
        try:
            id_poi=contents['@id']
            ids_poi.append(id_poi)
        except:
            ids_poi.append('NaN')
        
        try:
            photo= contents['hasMainRepresentation'][0]['ebucore:hasRelatedResource'][0]['ebucore:locator']
            photos.append(photo)
        except: 
            photos.append('NaN')
        
            
# Création du dataframe df 
df = pd.DataFrame({#'poi': labels,
    'ville': cities,
    #'code_postale': postal_codes,
    #'code_insee': codes_insee,
    'region': regions,
    'code_departement':code_regions,
   # 'departement': departements,
    'latitude': lats,
    'longitude': longs,
    'createur_donnée': office_tourismes,
    'fournisseur': fournisseurs,
    'id_poi':ids_poi,
    'photo': photos,
    'tag': tags_list                                
})

df

# Nettoyage des colonnes 
#df['departement']=df['departement'].apply(str).str.replace("[","").apply(str).str.replace("]","").apply(str).str.replace("'","")
df['region']=df['region'].apply(str).str.replace("[","").apply(str).str.replace("]","").apply(str).str.replace("'","")
df['photo']=df['photo'].str.replace("[","").str.replace("]","").str.replace("'","")
df['tag']=df['tag'].apply(str).str.replace("[","").apply(str).str.replace("]","").apply(str).str.replace("'","")


# Creation des catégories mères
df["categorie_mere"] = ((df["tag"].apply(lambda x: 'PlaceOfInterest' if 'PlaceOfInterest' in x else 'Product' if 'Product' in x else 'EntertainmentAndEvent' if 'EntertainmentAndEvent' in x else 'Tour')))


# Mettre la colonne 'photo' en booléen
df.photo[df.photo.notna()] = True
df.photo[df.photo.isna()] = False
df['photo']=df['photo'].astype(bool)


#Trouver dans la colonne 'tag' tous les 'schema:...' pour les effacer
df1= df['tag'].str.split(',',expand = True).stack().explode().value_counts().reset_index()
df1=df1.rename(columns={'index': 'cat'})
#df1

#trouver tous les 'schema:'
z=pd.DataFrame([(df1.cat.str.split(',',expand = True).stack().explode()),(df1.cat.str.split(',',expand = True).stack().explode().str.contains("schema").astype(str))]).transpose()

w=z[z.iloc[:,1] == 'True']

# La liste des "schema"
w=list(w[0])

df['tag2']=df['tag']

for i in w:
    df.tag2= df.tag2.str.replace(i+', ','')
    df.tag2= df.tag2.str.replace(i,'')
    
    
df.drop(['tag'], axis='columns', inplace= True)
df=df.rename(columns={'tag2': 'tag'})


# Fonction pour trouver les sous_catégories dans la colonne 'categorie_mere'
def categorie_tag(x):
    a= []
    # Création de la liste avec toutes les sous catégories
    liste_u= ["CyclingTour","FluvialOrSeaTour","RoadTour","UnderwaterTour","WalkingTour","HorseTour","Accommodation","BusinessPlace","MedicalPlace","NaturalHeritage","CulturalSite","FoodEstablishment","TastingProvider","ActivityProvider","TouristInformationCenter","Transport","ConvenientService","ServiceProvider","SportsAndLeisurePlace","Store","BusinessEvent","CulturalEvent","SaleEvent","SocialEvent","SportsAndRecreationEvent"]
    for i in liste_u:
    
        if i in x:
            
            a.append(i)
    return a


# Application de la fonction
df["sous_categorie"] = df["tag"].apply(categorie_tag)


# Nettoyage de la colonne 'sous_categorie'
df["sous_categorie"]=df["sous_categorie"].astype(str).str.replace("[","").astype(str).str.replace("]","").astype(str).str.replace("'","")

# Création du dataframe df_erreur_tag
df_erreur_tag=df[(df['categorie_mere']== 'Lieu') & (df.tag.str.contains('Event'))]

# Nettoyage des colonnes latitude et longitude
df['latitude']=df['latitude'].astype(float)
df['longitude']=df['longitude'].astype(float)
df['latitude']=df['latitude'].round(6)
df['longitude']=df['longitude'].round(6)

# Création du dataframe df_erreur_lat_lon
df_erreur_lat_lon = (df[(df.latitude.round(4) > 52)|(df.latitude.round(4) < 41)|(df.longitude.round(4) >10)|(df.longitude.round(4) <-6)])
df_erreur_lat_lon['len_dpt'] = df_erreur_lat_lon.code_departement.astype(str).apply(len)
df_erreur_lat_lon = df_erreur_lat_lon[df_erreur_lat_lon.len_dpt == 2]
df_erreur_lat_lon = df_erreur_lat_lon.iloc[:,0:-1]
    

# Retirer la colonne 'tag' pour alléger encore plus le dataframe
df.drop(['tag'], axis='columns', inplace= True)


def dept_clean(x):
    x = str(x)
    x = x.replace(".0","")
    if len(x) == 1:
        x = "0"+ x
    return x
df['code_departement'] = df.code_departement.apply(dept_clean)


# Traduction en français
#categorie_mere
df['categorie_mere']=df['categorie_mere'].str.replace("PlaceOfInterest","Lieu")
df['categorie_mere']=df['categorie_mere'].str.replace('Product','Produit')
df['categorie_mere']=df['categorie_mere'].str.replace('Tour','Itinéraire Touristique')
df['categorie_mere']=df['categorie_mere'].str.replace('EntertainmentAndEvent','Fête et Manifestation')

#sous_categorie
df['sous_categorie']=df['sous_categorie'].str.replace("BusinessEvent","Evenement Professionnel d'Entreprise")
df['sous_categorie']=df['sous_categorie'].str.replace('Accommodation','Hébergement')
df['sous_categorie']=df['sous_categorie'].str.replace('BusinessPlace',"Site d'Affaires")
df['sous_categorie']=df['sous_categorie'].str.replace('BusinessLieu',"Site d'Affaires")
df['sous_categorie']=df['sous_categorie'].str.replace('MedicalPlace','Lieu de Santé')
df['sous_categorie']=df['sous_categorie'].str.replace('MedicalLieu','Lieu de Santé')
df['sous_categorie']=df['sous_categorie'].str.replace('NaturalHeritage','Site Naturel')
df['sous_categorie']=df['sous_categorie'].str.replace('CulturalEvent','Evènement Culturel')
df['sous_categorie']=df['sous_categorie'].str.replace('CulturalSite','Site Culturel')
df['sous_categorie']=df['sous_categorie'].str.replace('FoodEstablishment','Restauration')
df['sous_categorie']=df['sous_categorie'].str.replace('EntertainmentAndEvent','Fête et manifestation')
df['sous_categorie']=df['sous_categorie'].str.replace('CyclingTour','Itinéraire cyclable')
df['sous_categorie']=df['sous_categorie'].str.replace('TastingProvider','Fournisseur de dégustation')
df['sous_categorie']=df['sous_categorie'].str.replace('FluvialOrSeaTour','Itinéraire fluvial ou maritime')
df['sous_categorie']=df['sous_categorie'].str.replace('RoadTour','Itinéraire routier')
df['sous_categorie']=df['sous_categorie'].str.replace('ActivityProvider',"Prestataire d'activité")
df['sous_categorie']=df['sous_categorie'].str.replace('UnderwaterTour','Itinéraire sous-marin')
df['sous_categorie']=df['sous_categorie'].str.replace('TouristInformationCenter',"Service d'information Touristique")
df['sous_categorie']=df['sous_categorie'].str.replace('Transport','Transport')
df['sous_categorie']=df['sous_categorie'].str.replace('ConvenientService','Prestataire de Service')
df['sous_categorie']=df['sous_categorie'].str.replace('WalkingTour','Itinéraire Pédestre')
df['sous_categorie']=df['sous_categorie'].str.replace('HorseTour','Itinéraire Equestre')
df['sous_categorie']=df['sous_categorie'].str.replace('SaleEvent','Evènement Commercial')
df['sous_categorie']=df['sous_categorie'].str.replace('SocialEvent','Evènement Social')
df['sous_categorie']=df['sous_categorie'].str.replace('SportsAndRecreationEvent','Evènement Sports et Loisirs')
df['sous_categorie']=df['sous_categorie'].str.replace('ServiceProvider','Prestataire de service')
df['sous_categorie']=df['sous_categorie'].str.replace('SportsAndLeisurePlace','Site Sportif, Récréatif et de Loisirs')
df['sous_categorie']=df['sous_categorie'].str.replace('SportsAndLeisureLieu','Site Sportif, Récréatif et de Loisirs')
df['sous_categorie']=df['sous_categorie'].str.replace('Store','Commerce de détail')
df['sous_categorie']=df['sous_categorie'].str.replace('Place','Lieu')
df['sous_categorie']=df['sous_categorie'].str.replace('Product','Produit')
df['sous_categorie']=df['sous_categorie'].str.replace('Visit','Visite')
df['sous_categorie']=df['sous_categorie'].str.replace('Visitee','Visite')
df['sous_categorie']=df['sous_categorie'].str.replace('Practice','Pratique')
df['sous_categorie']=df['sous_categorie'].str.replace('Rental','Location')

#Factorisation des villes
df['ville']=df['ville'].factorize()[0]

#Separation des dataframes en categorie_mere
df_lieu=df[df['categorie_mere']== 'Lieu']
df_produit=df[df['categorie_mere']== 'Produit']
df_it=df[df['categorie_mere']== 'Itinéraire Touristique']
df_fete=df[df['categorie_mere']== 'Fête et Manifestation']

# Charger les dataframe en CSV avec séparateur 
df.to_csv('df_tourisme.zip',sep=' ',index=False)
df_lieu.to_csv('df_lieu.zip',sep=' ',index=False)
df_produit.to_csv('df_produit.zip',sep=' ',index=False)
df_it.to_csv('df_it.zip',sep=' ',index=False)
df_fete.to_csv('df_fete.zip',sep=' ',index=False)
df_erreur_tag.to_csv('df_erreur_tag.csv',sep=' ',index=False)
df_erreur_lat_lon.to_csv('df_erreur_lat_lon.csv',sep=' ',index=False)